# Web Mining Project - LLM Expeimentation
* Fine tune Distil-Bert model
* Evaluate with Llamma 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%ls

drive/  sample_data/


In [ ]:
%cd '/content/drive/MyDrive/web_mining_project'

/content/drive/MyDrive/web_mining_project


In [ ]:
%ls

 dataset/               results/   train.csv                  web-mining-project.ipynb
'Project Report.gdoc'   test.csv   web-mining-llm-gpu.ipynb   wrong_preds.csv


## GPU

In [ ]:
!nvidia-smi

Mon May 13 11:46:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Fine tuning Distil-Bert model

In [ ]:
%pip install -qq -U datasets transformers[torch] accelerate==0.30.1 bitsandbytes==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.0 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset, Dataset
from datasets import load_metric
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,tweet,label
0,Talking Angela is a hoe,1
1,Lol they tricking niccas left n right,1
2,&#128514;&#128514; bitches get stuff done. htt...,1
3,@ArtaleGaspare bitch i got the flameeee,1
4,@vivaalakatyy @brotherbill7 @JHuff_5 good bitc...,1


In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class TextDataset(Dataset):
#   def __init__(self, encodings, labels):
#     self.encodings = encodings
#     self.labels = labels

#   def __len__(self):
#     return len(self.labels)

#   def __getitem__(self, idx):
#     item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
#     item['labels'] = torch.tensor(self.labels[idx])
#     return item

In [ ]:
# from sklearn.model_selection import train_test_split

# train_data, train_labels, val_data, val_labels = train_test_split(train_df['tweet'].toarray(),
#                                                                   train_df['labels'].toarray(),
#                                                                   test_size=0.2,
#                                                                   stratify=train_df['labels'].toarray())

In [ ]:
sentence_lengths = [len(x.split()) for x in train_df['tweet'].to_numpy()]
avg_len = np.mean(sentence_lengths)
max_len = np.max(sentence_lengths)
avg_len, max_len

(14.121557550691012, 33)

Loading the dataset

In [ ]:
dataset = load_dataset('csv',
                        data_files={'train': 'train.csv',
                                    'test': 'test.csv'},
                       trust_remote_code=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['train'][0]

{'tweet': 'Talking Angela is a hoe', 'label': 1}

Loading the Tokenizer and the bert model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(data):
  return tokenizer(data["tweet"], truncation=True, padding=True, max_length=max_len, return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# train_encodings = tokenizer(train_data, truncation=True, padding=True, max_length=256, return_tensors='pt')
# val_encodings = tokenizer(val_data, truncation=True, padding=True, max_length=256, return_tensors='pt')
# test_encodings = tokenizer(test_df['tweet'].toarray(), truncation=True, padding=True, max_length=256, return_tensors='pt')

In [ ]:
# train_dataset = TextDataset(train_encodings, train_labels)
# val_dataset = TextDataset(val_encodings, val_labels)
# test_dataset = TextDataset(test_encodings, test_df['labels'].toarray())

In [ ]:
dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/19826 [00:00<?, ? examples/s]

Map:   0%|          | 0/4957 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
id2label = {"0": "hate_speech", "1": "offensive_language", "2": 'neither'}
label2id = {"hate_speech": 0, "offensive_language": 1, 'neither': 2}

In [ ]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased",
                                                           num_labels=3,
                                                           id2label=id2label,
                                                           label2id=label2id).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation metrics

In [ ]:
def compute_metrics(eval_pred):
  load_f1 = load_metric("f1")
  load_accuracy = load_metric("accuracy")
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)
  f1_score = load_f1.compute(predictions=predictions, references=labels, average="weighted")
  return {"accuracy": accuracy, "f1-score": f1_score}

Fine tuning and evaluating bert model

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1-score
1,0.357000,0.282102,{'accuracy': 0.8952995763566673},{'f1': 0.8882046293097702}
2,0.260000,0.271435,{'accuracy': 0.9031672382489409},{'f1': 0.90085952477833}
3,0.223700,0.283616,{'accuracy': 0.9049828525317732},{'f1': 0.9001384646345959}


Trainer is attempting to log a value of "{'accuracy': 0.8952995763566673}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8882046293097702}" of type <class 'dict'> for key "eval/f1-score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9031672382489409}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.90085952477833}" of type <class 'dict'> for key "eval/f1-score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.9049828525317732}" of type <class 

CPU times: user 3min 26s, sys: 5.48 s, total: 3min 31s
Wall time: 5min 47s


TrainOutput(global_step=1860, training_loss=0.2654862762779318, metrics={'train_runtime': 346.7262, 'train_samples_per_second': 171.542, 'train_steps_per_second': 5.364, 'total_flos': 507828520945548.0, 'train_loss': 0.2654862762779318, 'epoch': 3.0})

In [ ]:
test_score_bert = trainer.evaluate()

Trainer is attempting to log a value of "{'accuracy': 0.9031672382489409}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.90085952477833}" of type <class 'dict'> for key "eval/f1-score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [ ]:
test_score_bert

{'eval_loss': 0.27143529057502747,
 'eval_accuracy': {'accuracy': 0.9031672382489409},
 'eval_f1-score': {'f1': 0.90085952477833},
 'eval_runtime': 5.9115,
 'eval_samples_per_second': 838.53,
 'eval_steps_per_second': 26.22,
 'epoch': 3.0}

In [ ]:
bert_preds = trainer.predict(dataset['test'])

In [ ]:
bert_labels = np.argmax(bert_preds.predictions, axis=-1)
bert_labels[:10]

### Custom evaluation function

In [ ]:
# evaluation function
from sklearn.metrics import precision_recall_fscore_support
from typing import Any

def evaluate(y_true: Any, y_pred: Any) -> dict[str, float]:
  accuracy = sum(y_true == y_pred) / len(y_true)
  precision, recall, f1_score, _ = precision_recall_fscore_support(y_true=y_true,
                                                                   y_pred=y_pred,
                                                                   average='weighted')
  return {'accuracy': round(accuracy*100, 2),
          'precision': round(precision*100, 2),
          'recall': round(recall*100, 2),
          'f1-score': round(f1_score*100, 2)}

In [ ]:
bert_score = evaluate(test_df['label'].values, bert_labels)
bert_score

{'accuracy': 90.32, 'precision': 89.91, 'recall': 90.32, 'f1-score': 90.09}

Making inference on wrong predictions from our initial model

In [ ]:
wrong_df = pd.read_csv('wrong_preds.csv')
wrong_df.head()

,tweet,true_label,pred_label,pred_prob,true_class,pred_class
0,RT @obeyyreggie: stfu hoe RT &#8220;@tedddydga...,0,1,0.790589,hate_speech,offensive_language
1,@jacksparenohoe let's hang out nigger,1,0,0.591132,offensive_language,hate_speech
2,@goodgraces_ ok faggot http://t.co/zdGtE9GvzX,1,0,0.657333,offensive_language,hate_speech
3,RT @HighSchoolSuckz: i am that horrible friend...,2,1,0.594451,neither,offensive_language
4,Noap. Not taking them. RT @TiffNCompany: Maca ...,2,1,0.684407,neither,offensive_language


In [ ]:
wrong_text = wrong_df[['tweet']]
wrong_dataset = Dataset.from_pandas(wrong_text)

In [ ]:
wrong_dataset = wrong_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/524 [00:00<?, ? examples/s]

In [ ]:
wrong_preds = trainer.predict(wrong_dataset)

In [ ]:
wrong_labels = np.argmax(wrong_preds.predictions, axis=-1)

In [ ]:
wrong_probs, _ = torch.max(torch.softmax(torch.tensor(wrong_preds.predictions), dim=-1), dim=-1)
wrong_probs[:10]

tensor([0.9787, 0.6424, 0.5418, 0.5648, 0.8276, 0.6816, 0.9384, 0.9668, 0.9493,
        0.4055])

In [ ]:
classes = ['hate_speech', 'offensive_language', 'neither']

In [ ]:
wrong_df['bert_pred'] = wrong_labels
wrong_df['bert_prob'] = wrong_probs
wrong_df['bert_class'] = [classes[i] for i in wrong_labels]

In [ ]:
most_wrong = wrong_df[wrong_df['true_label'] != wrong_df['bert_pred']].sort_values('bert_prob', ascending=False)
most_wrong[['tweet', 'true_class', 'pred_class', 'bert_class', 'pred_prob', 'bert_prob']].head(20)

,tweet,true_label,pred_label,pred_prob,true_class,pred_class,bert_pred,bert_prob
485,It's so shady when you bitches talk to guys w/...,0,1,0.985332,hate_speech,offensive_language,1,0.993381
478,No girls no hoes just me myself and I,2,1,0.978032,neither,offensive_language,1,0.993300
92,@JustDoItSlow &#128530;FOREVER 21 hoes,2,1,0.981271,neither,offensive_language,1,0.992928
419,RT @I_Be_kOoLz Food be good...except that rice...,2,1,0.976820,neither,offensive_language,1,0.992926
55,RT @Juicy_Squeaky: Any bitch who need a crowd ...,0,1,0.965192,hate_speech,offensive_language,1,0.992808
14,Lmao let these hoes be hoes ain't no Savin nem,0,1,0.991680,hate_speech,offensive_language,1,0.992637
235,money hoes and clothes,2,1,0.981338,neither,offensive_language,1,0.992499
268,RT @FriendlyAssh0le: Todays Horoscope: \n\nyou...,2,1,0.972252,neither,offensive_language,1,0.992224
78,RT @PacDaGoat: I really hate attention seeking...,0,1,0.824049,hate_speech,offensive_language,1,0.992176
259,and another one &#8220;@07Cannon: Two hoes? RT...,2,1,0.888212,neither,offensive_language,1,0.992140


Saving fine-tuned model

In [ ]:
# save_dir = './bert_saved_model'
# tokenizer.save_pretrained(save_dir)
# model.save_pretrained(save_dir)

Making predictions

In [ ]:
# fine_model = AutoModelForSequenceClassification.from_pretrained(save_dir).to(device)
# fine_tokenizer = AutoTokenizer.from_pretrained(save_dir)

In [ ]:
# batch = fine_tokenizer(test_df.tweet.tolist(), truncation=True, padding=True, max_length=256, return_tensors='pt')

In [ ]:
# with torch.inference_mode():
#   outputs = fine_model(**batch)
#   predictions = torch.argmax(outputs.logits, dim=-1)
#   pred_labels = [fine_model.config.id2label[prediction] for prediction in predictions.tolist()]

In [ ]:
# predictions[:10], pred_labels[:10]

## Llama 3

In [ ]:
# import pandas as pd
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')

In [ ]:
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

Getting huggingface token

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"

Quantisation Configuration

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Loading the Tokenizer and the Llama model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Model Pipeline

In [ ]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    device_map='auto'
)

Getting response

In [ ]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [ ]:
prompt_0 = """You are an expert linguist, who is good at classifying tweeter
    comments into hate_speech, offensive_language and neither labels.
    Help me classify tweeter comments into hate_speech(label=0), offensive_language(label=1) and neither(label=2).
    Classify the following sentence: """

In [ ]:
prompt = "Explain what data mining is."
# prompt = prompt_0 + 'Talking Angela is a hoe'

In [ ]:
response = get_response(prompt)
display(response[len(prompt):])

' What is the purpose of data mining? What is the relationship between data mining and data warehousing?\nExplain the difference between a data warehouse and a data mart. What are the advantages of a data warehouse? What are the advantages of a data mart?\nExplain the difference between a data warehouse and a data mart. What are the advantages of a data warehouse? What are the advantages of a data mart?\nExplain the difference between a data warehouse and a data mart. What are the advantages of a data warehouse? What are the advantages of a data mart?\nExplain the difference between a data warehouse and a data mart. What are'

In [ ]:
# def predictions(dataset):
#   results = []
#   for output in text_generator(KeyDataset(dataset, 'tweet')):
#     results.append(output)
#     print(output)
#   return results

In [ ]:
# results = predictions(dataset['test'])

## References

1. Hugging Face. "Meta-Llama 3 8B." Available at: [https://huggingface.co/meta-llama/Meta-Llama-3-8B](https://huggingface.co/meta-llama/Meta-Llama-3-8B).

2. Hugging Face. "DistilBERT base uncased." Available at: [https://huggingface.co/distilbert/distilbert-base-uncased](https://huggingface.co/distilbert/distilbert-base-uncased).

3. Hugging Face. "Datasets Documentation." Available at: [https://huggingface.co/docs/datasets/en/index](https://huggingface.co/docs/datasets/en/index).

4. Hugging Face. "Transformers Pipelines Documentation." Available at: [https://huggingface.co/docs/transformers/en/main_classes/pipelines](https://huggingface.co/docs/transformers/en/main_classes/pipelines).